<a href="https://colab.research.google.com/github/dpalominop/SistemasInteligentes/blob/master/semana15/homework/lstm-bi-text-generation-10k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test Generation using a Bidirectional LSTM Network

<br/>

In [1]:
from __future__ import print_function
#import Keras library
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, LambdaCallback
from keras.metrics import categorical_accuracy

import tensorflow as tf
#tf.device('/gpu:1')
from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())
#config = tf.ConfigProto(intra_op_parallelism_threads=4, \
#                        inter_op_parallelism_threads=4, \
#                        allow_soft_placement=True,\
#                        device_count = {'CPU' : 1, 'GPU' : 2})

#sess = tf.Session(config=config)
#K.set_session(sess)

#import spacy, and spacy french model
# spacy is used to work on text
#!pip install spacy
import spacy
#!python -m spacy download es
nlp = spacy.load('es')

#import other libraries
import numpy as np
import random
import sys
import os
import time
import codecs
import collections
from six.moves import cPickle
import pandas as pd

Using TensorFlow backend.


[]


In [2]:
gdrive = False
if gdrive:
    !pip install -U -q PyDrive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    # Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    
    !curl https://raw.githubusercontent.com/dexterfichuk/GoogleDriveCheckpoint/master/google_drive_checkpoint.py -O
      
    from google_drive_checkpoint import GoogleDriveCheckpoint

In [3]:
#define parameters used in the tutorial
data_dir = 'data/'# data directory containing raw texts
save_dir = 'models/' # directory to store trained NN models
file_list = ["spanish_emojis"]

if gdrive:
    link = "https://drive.google.com/open?id=1kdV9lTy7RNp4xYX6_IMA-6wfMRyXNSgv"
    fluff, id = link.split('=')
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('words_vocab.pkl')
    vocab_file = "words_vocab.pkl" 
    
    link = "https://drive.google.com/open?id=1xr0ZW6lBDTALEqaAdZefu-DwgEPIgM83"
    fluff, id = link.split('=')
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('words_list.pkl')
    words_list_file = "words_list.pkl" 
    
    link = "https://drive.google.com/open?id=17o5LV6AizHqPw9QeIjKwFXmTASpNs673"
    fluff, id = link.split('=')
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('spanish_emojis.csv')
    data_file = "spanish_emojis.csv"
    
    link = "https://drive.google.com/open?id=1gTUqnhuknwFP4BIFQixRIjB7V-CQyb_r"
    fluff, id = link.split('=')
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('sequences_list.pkl')
    sequences_file = "sequences_list.pkl" 
    
    link = "https://drive.google.com/open?id=13NEQdiBBn-3vCy8Q5JF-OBzrD7hs680j"
    fluff, id = link.split('=')
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('next_words_list.pkl')
    next_words_file = "next_words_list.pkl"
    
else:   
    vocab_file = os.path.join(data_dir, "words_vocab.pkl")
    words_list_file = os.path.join(data_dir, "words_list.pkl")
    data_file = os.path.join(data_dir, file_list[0] + ".csv")
    sequences_file = os.path.join(data_dir, "sequences_list.pkl")
    next_words_file = os.path.join(data_dir, "next_words_list.pkl")

sequences_step = 1 #step to create sequences
seq_length = 10 # sequence length
preprocess = False

In [9]:
def read_file(filename):
    items = []
    if os.path.exists(filename):
        # try:
        with open(filename, 'rb') as fname:
            while True:
                try:
                    items = cPickle.load(fname)
                except EOFError:
                    print(EOFError)
                    break
    else:
        items = []
    
    return items

In [5]:
def create_wordlist(doc):
    wl = []
    for word in doc:
        if word.text not in ("\n","\n\n",'\u2009','\xa0'):
            wl.append(word.text.lower())
    return wl

In [6]:
sequences = []
next_words = []
wordlist = []
if(preprocess):
    input_file = pd.read_csv(data_file)
    lines = input_file['observations']
    for data in lines:
        #create sentences
        doc = nlp(data)
        wl = create_wordlist(doc)
        wordlist = wordlist + wl
        if(len(wl) < 2): 
            continue
        seq = [' ' for i in range(seq_length - len(wl) + 1)]
        seq = seq + wl

        for i in range(0, len(seq) - seq_length, sequences_step):
          sequences.append(seq[i: i + seq_length])
          next_words.append(seq[i + seq_length])

In [10]:
if(preprocess):
    # count the number of words
    word_counts = collections.Counter(wordlist)

    # Mapping from index to word : that's the vocabulary
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    vocabulary_inv = list(sorted(vocabulary_inv))

    # Mapping from word to index
    vocab = {x: i for i, x in enumerate(vocabulary_inv)}
    words = [x[0] for x in word_counts.most_common()]

    #size of the vocabulary
    vocab_size = len(words)
    print("vocab size: ", vocab_size)

    #save the words and vocabulary
    with open(os.path.join(vocab_file), 'wb') as f:
        cPickle.dump((words, vocab, vocabulary_inv), f)
    with open(os.path.join(words_list_file), 'wb') as f:
        cPickle.dump((wordlist), f)
    with open(os.path.join(sequences_file), 'wb') as f:
        cPickle.dump((sequences), f)
    with open(os.path.join(next_words_file), 'wb') as f:
        cPickle.dump((next_words), f)

In [11]:
if(not preprocess):
    wordlist = read_file(words_list_file)
    (words, vocab, vocabulary_inv) = read_file(vocab_file)
    sequences = read_file(sequences_file)
    next_words = read_file(next_words_file)
    
    # count the number of words
    word_counts = collections.Counter(wordlist)
    
    #size of the vocabulary
    vocab_size = len(words)
    print("vocab size: ", vocab_size)

<class 'EOFError'>
<class 'EOFError'>
<class 'EOFError'>
<class 'EOFError'>
vocab size:  66846


In [9]:
#create sequences
#sequences = []
#next_words = []
#for i in range(0, len(wordlist) - seq_length, sequences_step):
#    sequences.append(wordlist[i: i + seq_length])
#    next_words.append(wordlist[i + seq_length])

print('nb sequences: ', len(sequences))
print('vocav( ): ', vocab[' '])
print('words(0): ', vocabulary_inv[0])

nb sequences:  421621
vocav( ):  0
words(0):   


In [10]:
def bidirectional_lstm_model(seq_length, vocab_size):
    print('Build LSTM model.')
    model = Sequential()
    model.add(Bidirectional(LSTM(rnn_size, activation="relu"),input_shape=(seq_length, vocab_size)))
    model.add(Dropout(0.2))
    model.add(Dense(vocab_size))
    model.add(Activation('softmax'))
    
    optimizer = Adam(lr=learning_rate)
    callbacks=[EarlyStopping(patience=2, monitor='val_loss')]
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[categorical_accuracy])
    return model

In [11]:
rnn_size = 40 # size of RNN
batch_size = 32 # minibatch size
seq_length = 10 # sequence length
num_epochs = 20 # number of epochs
learning_rate = 0.01 #learning rate
sequences_step = 1 #step to create sequences
BATCH_GEN = 100

In [12]:
md = bidirectional_lstm_model(seq_length, vocab_size)
md.summary()

Build LSTM model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 80)                21403840  
_________________________________________________________________
dropout_1 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 66846)             5414526   
_________________________________________________________________
activation_1 (Activation)    (None, 66846)             0         
Total params: 26,818,366
Trainable params: 26,818,366
Non-trainable params: 0
_________________________________________________________________


In [13]:
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, seq_length, len(words)), dtype=np.bool)
        y = np.zeros((batch_size, len(words)), dtype=np.bool)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index]):
                x[i, t, vocab[w]] = 1
            y[i, vocab[next_words[index]]] = 1

            index = index + 1
            if index == len(sentence_list):
                index = 0
        yield x, y

In [14]:
file_path = "BI-LSTM-epoch{epoch:03d}-words%d-sequence%d" % (
    len(words),
    seq_length
)
checkpoint = None
if gdrive:
    checkpoint = GoogleDriveCheckpoint(file_path, drive, save_best_only=True, save_weights_only=True, verbose=1, mode='auto', monitor='val_acc')
else:
    checkpoint = ModelCheckpoint(os.path.join(save_dir, file_path), monitor='val_acc', save_best_only=True)

#print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
early_stopping = EarlyStopping(monitor='val_acc', patience=5)
callbacks_list = [checkpoint, 
                  #print_callback, 
                  early_stopping]

In [15]:
with tf.device('/device:GPU:2'):
    md.fit_generator(
    generator(sequences[:10000], next_words[:10000], BATCH_GEN),
    steps_per_epoch=int(len(sequences[:10000])/BATCH_GEN) + 1,
    epochs=num_epochs,
    callbacks=callbacks_list,
    #validation_split=0.02,
    #shuffle=True,
    #validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),
    #validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1)
    )

Epoch 1/20
101/101 [==============================] - 76s 749ms/step - loss: 8.5630 - categorical_accuracy: 0.0236


D:\opt\Anaconda3\envs\SI\lib\site-packages\keras\callbacks.py:434: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
D:\opt\Anaconda3\envs\SI\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,categorical_accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/20
101/101 [==============================] - 70s 692ms/step - loss: 7.0832 - categorical_accuracy: 0.0272
Epoch 3/20
101/101 [==============================] - 70s 689ms/step - loss: 6.6337 - categorical_accuracy: 0.0397
Epoch 4/20
101/101 [==============================] - 69s 687ms/step - loss: 6.2011 - categorical_accuracy: 0.0554
Epoch 5/20
101/101 [==============================] - 69s 687ms/step - loss: 5.7995 - categorical_accuracy: 0.0775
Epoch 6/20
101/101 [==============================] - 70s 691ms/step - loss: 5.3744 - categorical_accuracy: 0.1032
Epoch 7/20
101/101 [==============================] - 70s 691ms/step - loss: 4.8181 - categorical_accuracy: 0.1414
Epoch 8/20
101/101 [==============================] - 70s 692ms/step - loss: 4.2414 - categorical_accuracy: 0.1802
Epoch 9/20
101/101 [==============================] - 70s 690ms/step - loss: 3.7431 - categorical_accuracy: 0.2412
Epoch 10/20
101/101 [==============================] - 69s 687ms/step - loss: 3.

In [16]:
#save the model
md.save(os.path.join(save_dir,  'my_model_gen_sentences_lstm2.final.hdf5'))

In [17]:
#load vocabulary
print("loading vocabulary...")
# vocab_file = os.path.join(save_dir, "words_vocab.pkl")

with open(vocab_file, 'rb') as f:
        words, vocab, vocabulary_inv = cPickle.load(f)

vocab_size = len(words)

loading vocabulary...


In [4]:
from keras.models import load_model
# load the model
print("loading model...")
model = load_model(save_dir + "/" + 'my_model_gen_sentences_lstm2.final.hdf5')

loading model...


In [5]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [21]:
#initiate sentences
#seed_sentences = "estoy muy cansado para estudiar hoy"
#seed_sentences = "todos vamos a"
seed_sentences = "para la tarea"
generated = ''
sentence = []
for i in range (seq_length):
    sentence.append(".")

seed = seed_sentences.split()

for i in range(len(seed)):
    sentence[seq_length-i-1]=seed[len(seed)-i-1]

generated += ' '.join(sentence)
print('Generating text with the following seed: "' + ' '.join(sentence) + '"')

print ()

Generating text with the following seed: ". . . . . . . para la tarea"



In [22]:
words_number = 20
#generate the text
for i in range(words_number):
    #create the vector
    x = np.zeros((1, seq_length, vocab_size))
    for t, word in enumerate(sentence):
        x[0, t, vocab[word]] = 1.
    #print(x.shape)

    #calculate next word
    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, 0.34)
    next_word = vocabulary_inv[next_index]

    #add the next word to the text
    generated += " " + next_word
    # shift the sentence by one, and and the next word at its end
    sentence = sentence[1:] + [next_word]

print(generated)

D:\opt\Anaconda3\envs\SI\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


. . . . . . . para la tarea me desperte a dormir para que ir a la nina que aun voy a dormir un ganas de irme a
